# define-market

In [1]:
symbols = 'SPY'
# symbols = ['SPY', 'GLD', 'SLV']

In [2]:
from quantopian.research.experimental import continuous_future

context = {}
context['markets'] = [
    continuous_future('US'),
    continuous_future('TY'),
    continuous_future('SB'),
    continuous_future('SF'),
    continuous_future('BP'),
    continuous_future('JY'),
    continuous_future('CD'),
    continuous_future('SP'),
    continuous_future('ED'),
    continuous_future('TB'),
    continuous_future('GC'),
    continuous_future('SV'),
    continuous_future('HG'),
    continuous_future('CL'),
    continuous_future('HO'),
    continuous_future('HU')
]

context['markets']

[ContinuousFuture(96074226540281856, root_symbol='US', offset=0, roll_style='volume', adjustment='mul'),
 ContinuousFuture(95799348633337856, root_symbol='TY', offset=0, roll_style='volume', adjustment='mul'),
 ContinuousFuture(95492584889188352, root_symbol='SB', offset=0, roll_style='volume', adjustment='mul'),
 ContinuousFuture(95496982935699456, root_symbol='SF', offset=0, roll_style='volume', adjustment='mul'),
 ContinuousFuture(90722903447896064, root_symbol='BP', offset=0, roll_style='volume', adjustment='mul'),
 ContinuousFuture(92984598866231296, root_symbol='JY', offset=0, roll_style='volume', adjustment='mul'),
 ContinuousFuture(90991184285073408, root_symbol='CD', offset=0, roll_style='volume', adjustment='mul'),
 ContinuousFuture(95507978051977216, root_symbol='SP', offset=0, roll_style='volume', adjustment='mul'),
 ContinuousFuture(91554134238494720, root_symbol='ED', offset=0, roll_style='volume', adjustment='mul'),
 ContinuousFuture(95774059865899008, root_symbol='TB', 

# compute-position-size

In [3]:
from datetime import datetime, timedelta

from pandas.tseries.offsets import BDay
from quantopian.research import get_pricing

PRICE_SLIDING_WINDOW = 57

end_date = datetime.now()
start_date = end_date - BDay(PRICE_SLIDING_WINDOW)

prices = get_pricing(
    symbols,
    start_date=start_date,
    end_date=end_date,
    fields=[
        'price',
        'high',
        'low',
        'close_price'
    ]
)

prices = prices.dropna()

prices.head()

,price,high,low,close_price
2017-02-21 00:00:00+00:00,235.426,235.665,234.490,235.426
2017-02-22 00:00:00+00:00,235.267,235.515,234.809,235.267
2017-02-23 00:00:00+00:00,235.416,235.874,234.540,235.416
2017-02-24 00:00:00+00:00,235.754,235.767,234.390,235.754
2017-02-27 00:00:00+00:00,236.103,236.282,235.326,236.103


In [4]:
from quantopian.research.experimental import history

futures_prices = history(
    context['markets'],
    fields=[
        'price',
        'high',
        'low',
        'close_price'
    ],
    frequency='daily',
    start_date='2017-05-03',
    end_date='2017-05-04'
)

futures_prices

<class 'pandas.core.panel.Panel'>
Dimensions: 4 (items) x 2 (major_axis) x 16 (minor_axis)
Items axis: price to close_price
Major_axis axis: 2017-05-03 00:00:00+00:00 to 2017-05-04 00:00:00+00:00
Minor_axis axis: ContinuousFuture(96074226540281856 [US, 0, volume, mul]) to ContinuousFuture(92417250866298880 [HU, 0, volume, mul])

In [5]:
for market in futures_prices.minor_axis:
    data_frame = futures_prices.minor_xs(market)
    
    print(data_frame)

                             price     high      low  close_price
2017-05-03 00:00:00+00:00  152.531  153.219  152.312      152.531
2017-05-04 00:00:00+00:00  151.750  152.594  151.344      151.750
                             price     high      low  close_price
2017-05-03 00:00:00+00:00  125.453  125.828  125.406      125.453
2017-05-04 00:00:00+00:00  125.172  125.484  125.062      125.172
                           price   high    low  close_price
2017-05-03 00:00:00+00:00  0.158  0.159  0.156        0.158
2017-05-04 00:00:00+00:00  0.154  0.158  0.153        0.154
                           price   high    low  close_price
2017-05-03 00:00:00+00:00  1.008  1.014  1.008        1.008
2017-05-04 00:00:00+00:00  1.016  1.016  1.007        1.016
                           price   high    low  close_price
2017-05-03 00:00:00+00:00  1.288  1.296  1.288        1.288
2017-05-04 00:00:00+00:00  1.294  1.295  1.285        1.294
                           price   high    low  close_price
2017

In [6]:
from talib import ATR

AVERAGE_TRUE_RANGE_SLIDING_WINDOW = 21
MOVING_AVERAGE = 20

volatility = ATR(
    prices.high[-AVERAGE_TRUE_RANGE_SLIDING_WINDOW:],
    prices.low[-AVERAGE_TRUE_RANGE_SLIDING_WINDOW:],
    prices.close_price[-AVERAGE_TRUE_RANGE_SLIDING_WINDOW:],
    timeperiod=MOVING_AVERAGE
)[-1]

volatility

1.3772499999999952

In [7]:
DOLLARS_PER_POINT = 1

dollar_volatility = volatility * DOLLARS_PER_POINT

dollar_volatility

1.3772499999999952

In [8]:
CAPITAL_PERCENTAGE_PER_TRADE = 0.01
START_CAPITAL = 1000000
CURRENT_CAPITAL = 900000
CAPITAL_MULTIPLIER = 2

profit = CURRENT_CAPITAL - START_CAPITAL
available_capital = START_CAPITAL

if profit < 0:
    available_capital = START_CAPITAL + profit * CAPITAL_MULTIPLIER

capital_per_trade = available_capital * CAPITAL_PERCENTAGE_PER_TRADE
trade_size = capital_per_trade / dollar_volatility

int(trade_size)

5808

# get-entry-signal

In [9]:
TWENTY_DAY_BREAKOUT = 20
FIFTY_FIVE_DAY_BREAKOUT = 55

In [10]:
twenty_day_high = prices.high[-TWENTY_DAY_BREAKOUT:].max()

twenty_day_high

240.19

In [11]:
fifty_five_day_high = prices.high[-FIFTY_FIVE_DAY_BREAKOUT:].max()

fifty_five_day_high

240.19

In [12]:
twenty_day_low = prices.low[-TWENTY_DAY_BREAKOUT:].min()

twenty_day_low

232.55000000000001

In [13]:
fifty_five_day_low = prices.low[-FIFTY_FIVE_DAY_BREAKOUT:].min()

fifty_five_day_low

231.61000000000001

In [14]:
latest_price = prices.price[-1]

latest_price

239.84999999999999

In [15]:
def get_system_one_entry_signal():
    """
    Get 20 day breakout entry signal.
    
    Does not factor in outcome of previous entry signal.
    """
    if latest_price > twenty_day_high:
        print('System one: Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < twenty_day_low:
        print('System one: Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('System one: No signal')
        
get_system_one_entry_signal()

System one: No signal


In [16]:
def get_system_two_entry_signal():
    """
    Get 55 day breakout entry signal.
    """
    if latest_price > fifty_five_day_high:
        print('System two: Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < fifty_five_day_low:
        print('System two: Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('System two: No signal')
        
get_system_two_entry_signal()

System two: No signal


In [17]:
latest_price = 240

get_system_one_entry_signal()

System one: No signal


In [18]:
latest_price = 232

get_system_one_entry_signal()

System one: Short SPY 5808@232.00


In [19]:
latest_price = 240

get_system_two_entry_signal()

System two: No signal


In [20]:
latest_price = 231

get_system_two_entry_signal()

System two: Short SPY 5808@231.00


In [21]:
def add_unit():
    """
    Add to position if price exceeds threshold.
    """
    previous_fill_price = 240

    long_threshold = previous_fill_price + volatility / 2
    short_threshold = previous_fill_price - volatility / 2

    if latest_price > long_threshold:
        print('Long %s %i@%.2f' % (symbols, trade_size, latest_price))
    elif latest_price < short_threshold:
        print('Short %s %i@%.2f' % (symbols, trade_size, latest_price))
    else:
        print('No signal')

In [22]:
latest_price = 241

add_unit()

Long SPY 5808@241.00


In [23]:
latest_price = 239

add_unit()

Short SPY 5808@239.00


In [24]:
latest_price = 239.5

add_unit()

No signal


# compute-stop

In [25]:
STOP_MULTIPLIER = 2

entry_price = 240
stop = entry_price - volatility * STOP_MULTIPLIER

stop

237.24550000000002

In [26]:
# Re-enter if price hits entry price after stopped out
STOP_MULTIPLIER = 0.5

entry_price = 240
stop = entry_price - volatility * STOP_MULTIPLIER

stop

239.311375

# get-exit-signal

In [27]:
TEN_DAY_EXIT_BREAKOUT = 10
TWENTY_DAY_EXIT_BREAKOUT = 20

In [28]:
ten_day_high = prices.high[-TEN_DAY_EXIT_BREAKOUT:].max()

ten_day_high

240.19

In [29]:
ten_day_low = prices.low[-TEN_DAY_EXIT_BREAKOUT:].min()

ten_day_low

237.69999999999999

In [30]:
twenty_day_high = prices.high[-TWENTY_DAY_EXIT_BREAKOUT:].max()

twenty_day_high

240.19

In [31]:
twenty_day_low = prices.low[-TWENTY_DAY_EXIT_BREAKOUT:].min()

twenty_day_low

232.55000000000001

In [32]:
def get_system_one_exit_signal():
    if position_type == 'long' and latest_price < ten_day_low \
            or position_type == 'short' and latest_price > ten_day_high:
        print('System one: Close %s' % symbols)
    else:
        print('System one: No signal')

In [33]:
def get_system_two_exit_signal():
    if position_type == 'long' and latest_price < twenty_day_low \
            or position_type == 'short' and latest_price > twenty_day_high:
        print('System two: Close %s' % symbols)
    else:
        print('System two: No signal')

In [34]:
position_type = 'long'
latest_price = 236

get_system_one_exit_signal()

System one: Close SPY


In [35]:
position_type = 'short'
latest_price = 240

get_system_one_exit_signal()

System one: No signal


In [36]:
position_type = 'long'
latest_price = 237

get_system_one_exit_signal()

System one: Close SPY


In [37]:
position_type = 'long'
latest_price = 232

get_system_two_exit_signal()

System two: Close SPY


In [38]:
position_type = 'short'
latest_price = 240

get_system_two_exit_signal()

System two: No signal


In [39]:
position_type = 'long'
latest_price = 233

get_system_two_exit_signal()

System two: No signal
